# GIS to get latitude and longitude for cities using python

### Change the parameters here

* Space_range - Spacing between points in 'metre'
* amtrix_size - Area to be covered. Value of 10 covers approx. 2.8 kms.

In [ ]:
space_range = 100       #in 'm'
matrix_size = 10   

In [1]:
#Import libraries
from geopy.geocoders import Nominatim
import folium
import json
import numpy as np
import requests
import pandas as pd
#import geopandas as gpd
import json


#### Create dataframe from the json file provided  - for the first country in list - Mexico

In [2]:
#**************** Read json file ********************
with open('city_list.json') as f:
    cities = json.load(f)
print(print(cities.keys()))
print(cities['data'].keys())
print(cities['data']['countryList'][0]['countryName'])
print(cities['data']['countryList'][0].keys())
print(cities['data']['countryList'][0]['cities'][0].keys())


dict_keys(['errno', 'errmsg', 'time', 'data', 'requestId', 'traceId'])
None
dict_keys(['countryList'])
Mexico
dict_keys(['countryCode', 'countryName', 'cities'])
dict_keys(['cityId', 'cityName', 'hotPoi'])


In [3]:
# **********  Form a dataframe from json **************

pd.set_option('display.max_rows', 10)
lstcity = []   #Initialize a list
id=0
for city in cities['data']['countryList'][0]['cities']:
    id += 1
    if 'name' in city['hotPoi']:
        nm = city['hotPoi']['name']
    else:
        nm=''
        
    lstcity.append([ (city['cityId'],
                      city['cityName'],
        city['hotPoi']['lat'],
        city['hotPoi']['lng'],
        nm
                     )])
    
dfcity = pd.DataFrame([item for lstcity in lstcity for item in lstcity])    #Create dataframe from list
dfcity.columns = ['CityId', 'CityName' , 'Latitude', 'Longitude', 'Address']
dfcity  = dfcity.sort_values(['Latitude'])

### Plot the hotpoints and city boundaries using folium map

In [4]:
#*********** plot the hotpoint of cities ************
#Let us first do it for 'Mexico' - get lat and long for 'mexico'

geolocator = Nominatim(user_agent = "myproj")
location = geolocator.geocode("Mexico")
conlat = location.latitude
conlng = location.longitude
print(conlat,conlng)

#show 'mexico'
map_city = folium.Map(location = [conlat, conlng],zoom_start =5 )

#Now mark the cities

#Add title to the map
title_html = '''<h3 align="center" style="font-size:20px"><b>
        GIS for Cities</b></h3>'''
map_city.get_root().html.add_child(folium.Element(title_html))

#*********** Now mark the boundaries of the cities *************
#Use geojson file for 'mexico' to draw outlies
folium.GeoJson("mexico.geojson", name = 'Mexico').add_to(map_city)

#Add markers that shows the hotpoints
for lat,lng,label in zip(dfcity['Latitude'],dfcity['Longitude'],dfcity['CityName']):
 
    label = folium.Popup(label,parse_html = True)
    
    folium.CircleMarker(
        [lat,lng],
        radius = 5,
        popup = label,
        color ='darkred',
        fill = True,
        fill_color = 'darkred',
        fill_opacity = 0.5,
        parse_html = False).add_to(map_city)
    
map_city

23.6585116 -102.0077097


### Generate equally spaced geo co-ordinates points

In [5]:
#Offset any lat long by x, y meters
def lat_long_offset(lat, lon, x, y):
    '''
    lat, lon : Provide lat lon coordinates
    x, y : Provide offset of x and y on lat and long respectively
           This needs to be in meters!
           
    The approximation is taken from an aviation formula from this stack exchange 
    https://gis.stackexchange.com/questions/2951/algorithm-for-offsetting-a-latitude-longitude-by-some-amount-of-meters
    '''
       
    #Earth’s radius, sphere
    R=6378137

    #Coordinate offsets in radians
    dLat = x/R
    dLon = y/(R*np.cos(np.pi*lat/180))

    #OffsetPosition, decimal degrees
    latO = lat + dLat * 180/np.pi
    lonO = lon + dLon * 180/np.pi
    
    return latO, lonO

#Create a vectorized offset function
lat_long_offset_vec = np.vectorize(lat_long_offset)


In [6]:
#Create offset_grid and return coordinates
def get_mesh(lat, lon, dist, coors):
    #calculate min and max range for coordinates over an axis
    mini, maxi = -dist*coors, dist*coors
    
    #calculate number of points over an axis
    n_coord = coors*2+1
    
    #create an axis from min to max value with required number of coordinates
    axis = np.linspace(mini, maxi, n_coord)
    
    #create an "offset_grid" for X and Y values for both axis. 
    X, Y = np.meshgrid(axis, axis)

    #calcualte offset coordinates for "offset_grid" in meters
    mesh = lat_long_offset_vec(lat, lon, X, Y)
    
    #Transpose to get the (x,y) values for the offset_grid's shape
    mesh_x_y_format = np.stack(mesh).transpose(1,2,0)
    return mesh_x_y_format



### Let us first plot for 'Aguascalientes'

In [7]:
#Let us start from centre of city 'Aguascalientes' - get its geo coordinates
dfcentre = dfcity.loc[dfcity['CityName'] == 'Aguascalientes']
clat = dfcentre['Latitude']
clng = dfcentre['Longitude']
print(clat,clng)

1    21.885234
Name: Latitude, dtype: float64 1   -102.291562
Name: Longitude, dtype: float64


In [21]:
#Get equally spaced grid for the given spec
output = get_mesh(clat,clng, space_range, matrix_size)
print( output)

-102.30124282731902
[[[  21.87625055 -102.30124283]
  [  21.87714886 -102.30124283]
  [  21.87804718 -102.30124283]
  [  21.87894549 -102.30124283]
  [  21.87984381 -102.30124283]
  [  21.88074212 -102.30124283]
  [  21.88164044 -102.30124283]
  [  21.88253875 -102.30124283]
  [  21.88343707 -102.30124283]
  [  21.88433538 -102.30124283]
  [  21.8852337  -102.30124283]
  [  21.88613202 -102.30124283]
  [  21.88703033 -102.30124283]
  [  21.88792865 -102.30124283]
  [  21.88882696 -102.30124283]
  [  21.88972528 -102.30124283]
  [  21.89062359 -102.30124283]
  [  21.89152191 -102.30124283]
  [  21.89242022 -102.30124283]
  [  21.89331854 -102.30124283]
  [  21.89421685 -102.30124283]]

 [[  21.87625055 -102.30027474]
  [  21.87714886 -102.30027474]
  [  21.87804718 -102.30027474]
  [  21.87894549 -102.30027474]
  [  21.87984381 -102.30027474]
  [  21.88074212 -102.30027474]
  [  21.88164044 -102.30027474]
  [  21.88253875 -102.30027474]
  [  21.88343707 -102.30027474]
  [  21.88433538 -

#### Loop thorugh the points and Plot only those points that has 'road' name

In [46]:
#print(output[5])
prefer_canvas=True
#Focus the map to the city
map_city = folium.Map(location = [output[0][0][0], output[0][0][1]],zoom_start = 15 )
cnt = 0

#Loop thorugh the points and plot the points that are placed in 'streets'
for a in range(len(output)):
    for b in range(len(output[a])):
        for c in range(len(output[a][b])):
            cnt +=1
            lat = output[a][b][0]
            lng = output[a][b][1]
            coord = str(lat) + ',' + str(lng)
            #print(coord)
            
            #Get address from latitude and longitude using reverse geocoding
            location = geolocator.reverse(coord,timeout=10)
            rawloc = location.raw
            
            #Check if the location has 'road' name 
            if 'address' in rawloc:
                if 'road' in rawloc['address']:
                    #Hovers the street name and coordinates
                    strl = coord + '-' + rawloc['address']['road']
                    label = folium.Popup(strl ,parse_html = True)

                    #Mark the points
                    folium.CircleMarker(
                        [lat, lng],
                        radius = 5,
                        color ='yellow',
                        popup = label,
                        fill = True,
                        fill_color = 'yellow',
                        fill_opacity = 0.5,
                        parse_html = False).add_to(map_city)

                    
map_city.save('Aguascalientes.html')

In [25]:
#Verifies distance between points
import geopy.distance
c1 = 21.87625055, -102.30124283
c2 = 21.89421685, -102.28284926
print(geopy.distance.distance(c1,c2).km)

2.7515477308929235


### Do it for 'Puebla'

In [44]:
#Let us start from centre of city 'Aguascalientes' - get its geo coordinates
dfcentre = dfcity.loc[dfcity['CityName'] == 'Puebla']
clat = dfcentre['Latitude']
clng = dfcentre['Longitude']
print(clat,clng)

#Get equally spaced grid for the given spec
output = get_mesh(clat,clng, space_range, matrix_size)


41    19.04144
Name: Latitude, dtype: float64 41   -98.206273
Name: Longitude, dtype: float64


In [45]:
#print(output[5])
prefer_canvas=True

#Focus the map to the city
map_city = folium.Map(location = [clat,clng],zoom_start =15 )
cnt = 0

#Loop thorugh the points and plot the points that are placed in 'streets'

for a in range(len(output)):
    for b in range(len(output[a])):
        for c in range(len(output[a][b])):
            cnt +=1
            lat = output[a][b][0]
            lng = output[a][b][1]
            coord = str(lat) + ',' + str(lng)
            #print(coord)
            
            #Get address from latitude and longitude using reverse geocoding
            location = geolocator.reverse(coord,timeout=10)
            rawloc = location.raw
            
            #Check if the location has 'road' name 
            if 'address' in rawloc:
                if 'road' in rawloc['address']:
                    strl = coord + '-' + rawloc['address']['road']
                    label = folium.Popup(strl ,parse_html = True)

                    #Mark the points
                    folium.CircleMarker(
                        [lat, lng],
                        radius = 5,
                        color ='yellow',
                        popup = label,
                        fill = True,
                        fill_color = 'yellow',
                        fill_opacity = 0.5,
                        parse_html = False).add_to(map_city)

                    
map_city.save('Puebla.html')